### Without Galtea

In [3]:
import requests
import json
from dotenv import load_dotenv

# Download the helper library from https://www.twilio.com/docs/python/install
import os
from twilio.rest import Client

load_dotenv()

# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = os.environ["TWILIO_ACCOUNT_SID"]
auth_token = os.environ["TWILIO_AUTH_TOKEN"]
client = Client(account_sid, auth_token)
# +34603829542, +34960324442
call = client.calls.create(
    from_="+12136957366",
    to="+34603829542",
    url="https://e2f2a707e179.ngrok-free.app/twilio-voice",
)

print(call.sid)

# --- Configuration ---
# Replace with the actual API key from your .env file
API_KEY = os.environ["API_KEY"] 
BASE_URL = "http://localhost:8001"

# --- Common Headers ---
headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json"
}

# --- Case 1: First call to get the initial user transcription ---
# This call will wait for the user to speak for the first time.
print("--- 1. Making the first call to get initial transcription (first=True) ---")

# Define parameters for the first call
# The 'input' parameter is ignored when 'first' is True, but we can include it.
params_first_call = {
    "first": True,
    "timeout": 50.0, # Wait up to 30 seconds
    "input": ""
}

try:
    response_first = requests.get(
        f"{BASE_URL}/generate",
        headers=headers,
        params=params_first_call
    )

    print(f"Status Code: {response_first.status_code}")

    if response_first.status_code == 200:
        print("Response JSON:", response_first.json())
    elif response_first.status_code == 204:
        print("Response: No content (Timeout reached, no user speech detected).")
    else:
        print("Error Response:", response_first.text)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")


print("\n" + "="*50 + "\n")


# --- Case 2: Subsequent call to send a response and get the next transcription ---
# This simulates sending your AI's audio response and waiting for the user's next message.
print("--- 2. Making a subsequent call to send a response (first=False) ---")

# Define parameters for a subsequent call
params_subsequent_call = {
    "first": False,
    "timeout": 50.0,
    "input": "No, no quiero escuchar las condiciones. ¿Puedes, por favor, seguir adelante?."
}

try:
    response_subsequent = requests.get(
        f"{BASE_URL}/generate",
        headers=headers,
        params=params_subsequent_call
    )

    print(f"Status Code: {response_subsequent.status_code}")

    if response_subsequent.status_code == 200:
        print("Response JSON:", response_subsequent.json())
    elif response_subsequent.status_code == 204:
        print("Response: No content (Timeout reached, no new user speech detected).")
    else:
        print("Error Response:", response_subsequent.text)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")


# --- Case 3: Subsequent call to send a response and get the next transcription ---
# This simulates sending your AI's audio response and waiting for the user's next message.
print("--- 3. Making a subsequent call to send a response (first=False) ---")

# Define parameters for a subsequent call
params_subsequent_call = {
    "first": False,
    "timeout": 50.0,
    "input": "¿Podrías contarme qué tipo de servicios ofreces?"
}

try:
    response_subsequent = requests.get(
        f"{BASE_URL}/generate",
        headers=headers,
        params=params_subsequent_call
    )

    print(f"Status Code: {response_subsequent.status_code}")

    if response_subsequent.status_code == 200:
        print("Response JSON:", response_subsequent.json())
    elif response_subsequent.status_code == 204:
        print("Response: No content (Timeout reached, no new user speech detected).")
    else:
        print("Error Response:", response_subsequent.text)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")




CA7f6043ddd0e7ad0c459cbf43314f142e
--- 1. Making the first call to get initial transcription (first=True) ---
Status Code: 200
Response JSON: {'response': 'Hola, ¿qué tal? Espero que... mucho tiempo tengo y... (pausa de seis segundos)'}


--- 2. Making a subsequent call to send a response (first=False) ---
Status Code: 200
Response JSON: {'response': 'Sí, claro. Podemos hablar adelante si tú quieres. Ah, cualquier cosa tú quieres. Mmm, ah. Si acabo de... si acabo de llegar.'}
--- 3. Making a subsequent call to send a response (first=False) ---


KeyboardInterrupt: 

In [4]:
result = client.calls(f"{call.sid}").update(status='completed')
result 

<Twilio.Api.V2010.CallInstance account_sid=AC0879f783274ebb3e6402b0c7e540ac28 sid=CA7f6043ddd0e7ad0c459cbf43314f142e>

### With Galtea

In [ ]:
import galtea
import requests
import json
from dotenv import load_dotenv
import os
from twilio.rest import Client

load_dotenv()

class MyAgent(galtea.Agent):
    def __init__(self,remote_url,from_number,to_number):
        account_sid = os.environ["TWILIO_ACCOUNT_SID"]
        auth_token = os.environ["TWILIO_AUTH_TOKEN"]
        self.client = Client(account_sid, auth_token)
        API_KEY = os.environ["API_KEY"] 
        self.BASE_URL = "http://localhost:8001"
        self.headers = { "x-api-key": API_KEY, "Content-Type": "application/json" }
        self.remote_url = remote_url
        self.from_number = from_number
        self.to_number = to_number

    def start_call(self):
        
        self.call_twilio = self.client.calls.create(
            from_=self.from_number,
            to=self.to_number,
            url=f"{self.remote_url}/twilio-voice",
        )
        print(f"Call started with sid: {self.call_twilio.sid}")

    def end_call(self):
        self.client.calls(f"{self.call_twilio.sid}").update(status='completed')

    def generate_(self,first, timeout, input):
        params_first_call = { "first": first,  "timeout": timeout, "input": input }
        try:
            response_first = requests.get( f"{self.BASE_URL}/generate",  headers=self.headers, params=params_first_call )
            if response_first.status_code == 200:
                print("Response JSON:", response_first.json())
                return response_first.json()
            elif response_first.status_code == 204:
                print("Response: No content (Timeout reached, no user speech detected).")
            else:
                print("Error Response:", response_first.text)
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")


    def call(self, input_data: galtea.AgentInput) -> galtea.AgentResponse:

        if len(input_data.messages) ==0:
            self.start_call()
            print("call started")
            response = self.generate_(True, 30.0, "")
            print("response generated")
        else:
            print("generating response with user message")
            user_message = input_data.last_user_message_str()
            print("generating response with user message1")

            response = self.generate_(False, 30.0, user_message)
            print("generating response with user message2")


        return galtea.AgentResponse( content=response["response"],  metadata={"model_version": "1.0"} )


In [8]:
from galtea import Galtea
from dotenv import load_dotenv
import os,uuid,time

load_dotenv()

api_key = os.environ["GALTEA_API_KEY_DEV"]
galtea_client = Galtea(api_key=api_key)
test_cases = galtea_client.test_cases.list(test_id="mayfgsdeqpc076x36l5ckymz")


results = []
test_case_num = [6,7,8]     
for i, test_case in enumerate(test_cases):
    if i in test_case_num:

        time.sleep(2)
        # +34603829542, +34960324442
        agent = MyAgent(remote_url="https://6e2c64d82771.ngrok-free.app",
        from_number="+12136957366",to_number="+34960324442")
        uid = str(uuid.uuid4())

        # Create a session for this test case
        session = galtea_client.sessions.create(
            version_id="hwtq29i0jrhosqpukiobu6n4",
            test_case_id=test_case.id,
            custom_id=f"Phone{uid}" # got stuck on Blocking1
        )

        # Run the simulation
        result = galtea_client.simulator.simulate(
            session_id=session.id,
            agent=agent,
            max_turns=12,
            agent_goes_first=True
        )
        print("call ended")

        agent.end_call()
        results.append(result)

        # Review results
        print(f"Completed {result.total_turns} turns. Finished: {result.finished}")
        if result.stopping_reason:
            print(f"Stopping reason: {result.stopping_reason}")
        print(f"passed one thing_, {i}")
        # break 

        

Starting conversation simulation...
Call started with sid: CAb5a73dd93385a385bcca32dba7892b6a
call started
Response JSON: {'response': 'Buenas tardes. Soy María, asistente de Premium Health and Sport. Para mejorar su atención, la llamada va a ser grabada. Si desea conocer la política de protección de datos, puede indicármelo y se las leeré. ¿Desea escuchar las condiciones de protección de datos? Si no quiere escucharlas, se asume que está aceptando las condiciones automáticamente.'}
response generated
generating response with user message
generating response with user message1
Response JSON: {'response': 'Un momento mientras lo reviso (teclea). No dispongo de información específica sobre el mostrador al que debe acudir al llegar. Si necesita indicaciones concretas, le recomiendo preguntar al personal al ingresar, quienes le guiarán en recepción. ¿Le gustaría saber algo más sobre nuestras instalaciones o servicios? (pausa de dos segundos)'}
generating response with user message2
generat

Agent call failed on turn 7: 'NoneType' object is not subscriptable


Response: No content (Timeout reached, no user speech detected).
generating response with user message2
call ended
Completed 7 turns. Finished: True
Stopping reason: Agent error: 'NoneType' object is not subscriptable
passed one thing_, 6
Starting conversation simulation...
Call started with sid: CA124aa94188c6fc71859dc9de577d466a
call started
Response JSON: {'response': 'Buenas tardes. Soy María, asistente de Premium Health and Sport. Para mejorar su atención, la llamada va a ser grabada. Si desea conocer la política de protección de datos, puede indicármelo y se las leeré (pausa de cuatro segundos).'}
response generated
generating response with user message
generating response with user message1
Response JSON: {'response': 'De datos, si no desea escucharlas, se asumirá que las acepta (pausa de tres segundos).'}
generating response with user message2
generating response with user message
generating response with user message1
Response JSON: {'response': 'Antes de poder continuar y pro

Agent call failed on turn 10: 'NoneType' object is not subscriptable


Response: No content (Timeout reached, no user speech detected).
generating response with user message2
call ended
Completed 10 turns. Finished: False
Stopping reason: Agent error: 'NoneType' object is not subscriptable
passed one thing_, 7
Starting conversation simulation...
Call started with sid: CA1d7fe0754313cf997dc5f9d26882e19d
call started
Response JSON: {'response': 'Buenas tardes. Soy María, asistente de Premium Health and Sport. Para mejorar su atención, la llamada va a ser grabada. Si desea conocer la política de protección de datos, puede indicármelo y se las leeré. ¿Desea que le lea las condiciones de protección de datos? Si no desea escucharlas, se considerará que las acepta para poder continuar con la gestión.'}
response generated
generating response with user message
generating response with user message1
Response JSON: {'response': 'Vale. Antes de continuar, ¿desea que le lea las condiciones de protección de datos o prefiere continuar y se considerarán aceptadas? Es nec

In [5]:
sid = "CA6d1f829936994b37d54d98926213a272"

load_dotenv()
account_sid = os.environ["TWILIO_ACCOUNT_SID"]
auth_token = os.environ["TWILIO_AUTH_TOKEN"]
client = Client(account_sid, auth_token)

resulting = client.calls(f"{sid}").update(status='completed')
resulting 

<Twilio.Api.V2010.CallInstance account_sid=AC0879f783274ebb3e6402b0c7e540ac28 sid=CA6d1f829936994b37d54d98926213a272>

In [12]:
resulting()

TypeError: 'CallInstance' object is not callable

### Listing Twilio

In [6]:
from twilio.rest import Client
import os

# Your Account SID and Auth Token from twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = os.environ['TWILIO_ACCOUNT_SID']
auth_token = os.environ['TWILIO_AUTH_TOKEN']

client = Client(account_sid, auth_token)

calls = client.calls.list(status="in-progress")

for record in calls:
    print(record.sid)

In [11]:
from twilio.rest import Client
import os

# Your Account SID and Auth Token from twilio.com/console
account_sid = os.environ['TWILIO_ACCOUNT_SID']
auth_token = os.environ['TWILIO_AUTH_TOKEN']

client = Client(account_sid, auth_token)

# List all completed calls
calls = client.calls.list(status="completed")

# Iterate through the calls and print their details
for record in calls:
    print(f"Call SID: {record.sid}")
    print(f"From: {record.from_formatted}")
    print(f"To: {record.to_formatted}")
    print(f"Start Time: {record.start_time}")
    print(f"End Time: {record.end_time}")
    print(f"Duration: {record.duration} seconds\n")

Call SID: CA7b55a5f941b70299e335a9a88862d43b
From: (213) 695-7366
To: +34960324442
Start Time: 2025-09-10 20:05:31+00:00
End Time: 2025-09-10 20:05:54+00:00
Duration: 23 seconds

Call SID: CAf3f7784300065b9fa0a33d065d435f63
From: (213) 695-7366
To: +34603829542
Start Time: 2025-09-10 20:03:06+00:00
End Time: 2025-09-10 20:03:09+00:00
Duration: 3 seconds

Call SID: CAcab1259e6db5373493c326acddd5e1ee
From: (213) 695-7366
To: +34603829542
Start Time: 2025-09-10 20:01:21+00:00
End Time: 2025-09-10 20:01:28+00:00
Duration: 7 seconds

Call SID: CAd46617dfeb6c85ee88537a76288b81a5
From: (213) 695-7366
To: +34603829542
Start Time: 2025-09-10 19:59:07+00:00
End Time: 2025-09-10 19:59:08+00:00
Duration: 1 seconds

Call SID: CA3cd9ec8d8fe47b9ad1e4c5a269548e70
From: (213) 695-7366
To: +34603829542
Start Time: 2025-09-10 19:56:50+00:00
End Time: 2025-09-10 19:56:59+00:00
Duration: 9 seconds

Call SID: CA5f4424cbff5a8715e9f768151fb9192d
From: (213) 695-7366
To: +34603829542
Start Time: 2025-09-10 19:

### 11Labs

In [87]:
# example.py
import os
from dotenv import load_dotenv
from io import BytesIO
import requests
from elevenlabs.client import ElevenLabs
from elevenlabs import VoiceSettings
import audioop
load_dotenv()

elevenlabs = ElevenLabs(
  api_key=os.getenv("ELEVENLABS_API_KEY_GAL"),
)

local_wav_file_path = "/Users/akshay/Downloads/Galtea/phone/recording_1757600922.wav" # <--- IMPORTANT: Replace with your actual file path

# 2. Open the local WAV file in binary read mode ('rb')

# with open(local_wav_file_path, 'rb') as audio_file:
#     # 3. Read the content and wrap it in BytesIO, or pass the file object directly
#     # Passing the file object directly is often more efficient and cleaner:
#     transcription = elevenlabs.speech_to_text.convert(
#         file=audio_file, # Pass the file object directly
#         model_id="scribe_v1",
#         tag_audio_events=True,
#         language_code="es",
#         diarize=True,
#     )

#     print(transcription.text)

user_response = "This is a test response from the server"
response = elevenlabs.text_to_speech.convert(
                    voice_id="5IDdqnXnlsZ1FCxoOFYg",  output_format="pcm_16000", text=user_response,  model_id="eleven_flash_v2_5",language_code="es" ,
                    voice_settings=VoiceSettings( stability=0.0, similarity_boost=1.0, style=1.0, use_speaker_boost=True, speed=1.0, ), )
audio_buffer = b''
for chunk in response:
    if chunk:
        audio_buffer += chunk                
mulaw_bytes = audioop.lin2ulaw(audio_buffer, 2)  # width=2 for 16-bit PCM

In [88]:
mulaw_bytes

b'\xfd\xfc\xfc\xfd\xfd\xfd\xfc\xfd\xfc\xfc\xfc\xfc\xfc\xfc\xfb\xfc\xfc\xfc\xfd\xfd\xfd\xfd\xfd\xfe\xfe\xfe\xfe\xff\xff\xff~~~}}}}}}}}~~~}}~}~~~}}}}}|||||}}||||||}|}||}}|}}}}}~}~~~~}}~~~~~~~~}~}}}}}}}}}|}}}}}}}}}}||||}}}|}}}}}||||}}}}}~~~~\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xfe\xfe\xfe\xfd\xfd\xfc\xfc\xfc\xfc\xfc\xfb\xfb\xfb\xfa\xfa\xfa\xfa\xfa\xfa\xf9\xf9\xf9\xf9\xf9\xf9\xf9\xfa\xfa\xfa\xfa\xfb\xfb\xfb\xfb\xfc\xfc\xfc\xfc\xfd\xfe\xfe\xfe\xff\xff\xff~~}}}}}}|||{{zyxwwvuutsrqpooonnnnmmmmnnnnooopqrstvxz|~\xfd\xfa\xf7\xf4\xf2\xef\xee\xec\xeb\xe9\xe8\xe6\xe5\xe3\xe2\xe0\xdf\xdf\xde\xde\xde\xdd\xdd\xdd\xdc\xdc\xdd\xdd\xdd\xdd\xdd\xdd\xde\xde\xde\xdf\xdf\xe0\xe2\xe3\xe4\xe6\xe7\xe9\xea\xec\xee\xf1\xf5\xfa\xfe{uplifc`^\\ZXVTSQPONNMMMLLLLLMMMMNNOPQSUWY[^bglr}\xf6\xed\xe8\xe4\xe0\xde\xdc\xda\xd8\xd7\xd5\xd4\xd2\xd1\xcf\xce\xcd\xcc\xcb\xcb\xca\xc9\xc8\xc8\xc7\xc7\xc7\xc7\xc6\xc6\xc7\xc7\xc7\xc8\xc8\xc9\xca\xcb\xcc\xce\xcf\xd0\xd3\xd6\xd9\xdc\xdf\xe6\xec\xf4}qjd_]ZXUSPONMLKJIIHGFFEDDCCBBA

  ### Next